In [ ]:
%pip install -r requirements.txt

In [ ]:
%pip install "git+https://github.com/apache/iceberg-python.git#egg=pyiceberg[pyarrow]"

In [1]:
import duckdb
from pyiceberg.catalog.sql import SqlCatalog

warehouse_path = "./icehouse"
catalog = SqlCatalog("default",**{"uri": "sqlite:///:memory:", "warehouse": f"file://{warehouse_path}"})
catalog.create_namespace("test_ns")

df = duckdb.sql("""
        select 1 as order_id, 'online' as order_type, 23.99 as price
        union all
        select 2 as order_id, 'in-store' as order_type, 34.99 as price
        """).arrow()

table = catalog.create_table("test_ns.orders", schema=df.schema)
table.append(df)

In [2]:
cn = table.scan().to_duckdb("orders")
cn.sql(f"select * from orders").show()

┌──────────┬────────────┬──────────────┐
│ order_id │ order_type │    price     │
│  int32   │  varchar   │ decimal(4,2) │
├──────────┼────────────┼──────────────┤
│        1 │ online     │        23.99 │
│        2 │ in-store   │        34.99 │
└──────────┴────────────┴──────────────┘



In [3]:
df2 = duckdb.sql("""
        select 3 as order_id, 'online' as order_type, 46.55 as price
        union all
        select 2 as order_id, 'in-store' as order_type, 27.81 as price
        """).arrow()
print(df2)

pyarrow.Table
order_id: int32
order_type: string
price: decimal128(4, 2)
----
order_id: [[3],[2]]
order_type: [["online"],["in-store"]]
price: [[46.55],[27.81]]


In [8]:
table.upsert(df2, ["order_id"])

UpsertResult(rows_updated=1, rows_inserted=1)

In [16]:
cn = table.scan().to_duckdb("orders")
cn.sql(f"select * from orders").show()

┌──────────┬────────────┬──────────────┐
│ order_id │ order_type │    price     │
│  int32   │  varchar   │ decimal(4,2) │
├──────────┼────────────┼──────────────┤
│        3 │ online     │        46.55 │
│        2 │ in-store   │        27.81 │
│        1 │ online     │        23.99 │
└──────────┴────────────┴──────────────┘

